# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [2]:
# Your code here

import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [3]:
q = ("""
SELECT firstName AS "First Name", lastName AS "Last Name", city AS "Employee City"
FROM employees
JOIN offices
USING(officeCode)
WHERE city = 'Boston'
""")

pd.read_sql(q,conn)

,First Name,Last Name,Employee City
0,Julie,Firrelli,Boston
1,Steve,Patterson,Boston


In [4]:
pd.read_sql("""          
SELECT * 
FROM offices       
""",conn).columns

Index(['officeCode', 'city', 'phone', 'addressLine1', 'addressLine2', 'state',
       'country', 'postalCode', 'territory'],
      dtype='object')

In [5]:
pd.read_sql("""          
SELECT * 
FROM employees       
""",conn).columns

Index(['employeeNumber', 'lastName', 'firstName', 'extension', 'email',
       'officeCode', 'reportsTo', 'jobTitle'],
      dtype='object')

## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [6]:
# Your code here
q = ("""
SELECT officeCode AS "Office Code", 
       city AS "Office City",
       COUNT(DISTINCT employeeNumber) AS "Employees"
FROM offices
LEFT JOIN employees
USING(officeCode)
GROUP BY officeCode
HAVING COUNT(employeeNumber) = 0
""")

pd.read_sql(q,conn)

,Office Code,Office City,Employees
0,27,Boston,0


## Write 3 questions of your own and answer them

In [7]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""
q = ("""
SELECT offices.officeCode AS "Office Code",
       offices.city AS "Office City",
       COUNT(DISTINCT customers.customerNumber) AS "Num Customers"
FROM offices
LEFT JOIN employees
    ON offices.officeCode = employees.officeCode
LEFT JOIN customers
    ON employees.employeeNumber = customers.salesRepEmployeeNumber
GROUP BY offices.officeCode, offices.city

     
"""
)

pd.read_sql(q,conn)

,Office Code,Office City,Num Customers
0,1,San Francisco,12
1,2,Boston,12
2,3,NYC,15
3,4,Paris,29
4,5,Tokyo,5
5,6,Sydney,10
6,7,London,17
7,27,Boston,0


In [8]:
"""
Question 1
Which customers have no payments at all?
"""

q = ("""
SELECT
    customers.customerName AS "Customer Name",
    payments.paymentDate AS "Payment Date",
    payments.amount AS "Payment Amount"
FROM customers
LEFT JOIN payments
USING(customerNumber)
WHERE (paymentDate IS NULL) AND (amount IS NULL)
"""
)

pd.read_sql(q,conn)



# Your code here

,Customer Name,Payment Date,Payment Amount
0,Havel & Zbyszek Co,None,None
1,American Souvenirs Inc,None,None
2,Porto Imports Co.,None,None
3,"Asian Shopping Network, Co",None,None
4,Natürlich Autos,None,None
5,ANG Resellers,None,None
6,Messner Shopping Network,None,None
7,"Franken Gifts, Co",None,None
8,BG&E Collectables,None,None
9,Schuyler Imports,None,None


In [9]:
"""
2. Which product has generated the most revenue overall?
"""

q = ("""
SELECT
    products.productCode AS "Product Code",
    products.productName AS "Product Name",
    SUM(orderdetails.quantityOrdered * orderdetails.priceEach) AS "Revenue"
FROM products
LEFT JOIN orderdetails
USING(productCode)
GROUP BY productCode, productName
ORDER BY "Revenue" DESC
LIMIT 1
"""
)

pd.read_sql(q,conn)


,Product Code,Product Name,Revenue
0,S18_3232,1992 Ferrari 360 Spider red,276839.98


In [10]:
"""
Which employee is associated with the most customers?
"""

q = ("""
SELECT
    employees.employeeNumber AS "Employee Number",
    employees.firstName || ' ' || employees.lastName AS "Employee Name",
    COUNT(customerNumber) AS "Customer Count"
FROM employees
LEFT JOIN customers
    ON employees.employeeNumber = customers.salesRepEmployeeNumber
GROUP BY employees.employeeNumber, "Employee Name"
ORDER BY COUNT(customerNumber) DESC
LIMIT 1      
""")

pd.read_sql(q,conn)

,Employee Number,Employee Name,Customer Count
0,1401,Pamela Castillo,10


In [11]:
"""
Which office has the highest total order value across all its employees’ customers?
"""

q = ("""
SELECT offices.officeCode,
       offices.city,
       SUM(orderdetails.priceEach * orderdetails.quantityOrdered) AS total_office_sales
FROM offices
JOIN employees
  ON offices.officeCode = employees.officeCode
JOIN customers
  ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders
  ON customers.customerNumber = orders.customerNumber
JOIN orderdetails
  ON orders.orderNumber = orderdetails.orderNumber
GROUP BY offices.officeCode
ORDER BY total_office_sales DESC
LIMIT 1;
""")

pd.read_sql(q,conn)


,officeCode,city,total_office_sales
0,4,Paris,3083761.58


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [12]:
# Your code here
"""
product --> orderdetails ---> orders ---> customers ---> employees
"""

q = ("""
SELECT
    products.productCode AS "Product Code",
    products.productName AS "Product Name",
    employees.employeeNumber AS "Employe Number",
    employees.firstName || ' ' || employees.lastName AS "Employee Name"
FROM products
JOIN orderdetails
    ON products.productCode = orderdetails.productCode
JOIN orders
    ON orderdetails.orderNumber = orders.orderNumber
JOIN customers
    ON orders.customerNumber = customers.customerNumber
JOIN employees
    ON customers.salesRepEmployeeNumber = employees.employeeNumber 
    
""")

pd.read_sql(q,conn)


,Product Code,Product Name,Employe Number,Employee Name
0,S10_1678,1969 Harley Davidson Ultimate Chopper,1323,George Vanauf
1,S10_1678,1969 Harley Davidson Ultimate Chopper,1337,Loui Bondur
2,S10_1678,1969 Harley Davidson Ultimate Chopper,1337,Loui Bondur
3,S10_1678,1969 Harley Davidson Ultimate Chopper,1166,Leslie Thompson
4,S10_1678,1969 Harley Davidson Ultimate Chopper,1165,Leslie Jennings
...,...,...,...,...
2991,S72_3212,Pont Yacht,1612,Peter Marsh
2992,S72_3212,Pont Yacht,1501,Larry Bott
2993,S72_3212,Pont Yacht,1370,Gerard Hernandez
2994,S72_3212,Pont Yacht,1370,Gerard Hernandez


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [13]:
q_products = ("""
SELECT * 
FROM products     
""")

q_orderdetails = ("""
SELECT *
FROM orderdetails
""")
q_orders = ("""
SELECT *
FROM orders
""")
q_customers = ("""
SELECT * 
FROM customers
""")
q_employees = ("""
SELECT *
FROM employees
""")

print(pd.read_sql(q_products,conn).columns, ("products").upper())
print(pd.read_sql(q_orderdetails,conn).columns, ("orderdetails").upper())
print(pd.read_sql(q_orders,conn).columns, ("orders").upper())
print(pd.read_sql(q_customers,conn).columns, ("customers").upper())
print(pd.read_sql(q_employees,conn).columns, ("employees").upper())

Index(['productCode', 'productName', 'productLine', 'productScale',
       'productVendor', 'productDescription', 'quantityInStock', 'buyPrice',
       'MSRP'],
      dtype='object') PRODUCTS
Index(['orderNumber', 'productCode', 'quantityOrdered', 'priceEach',
       'orderLineNumber'],
      dtype='object') ORDERDETAILS
Index(['orderNumber', 'orderDate', 'requiredDate', 'shippedDate', 'status',
       'comments', 'customerNumber'],
      dtype='object') ORDERS
Index(['customerNumber', 'customerName', 'contactLastName', 'contactFirstName',
       'phone', 'addressLine1', 'addressLine2', 'city', 'state', 'postalCode',
       'country', 'salesRepEmployeeNumber', 'creditLimit'],
      dtype='object') CUSTOMERS
Index(['employeeNumber', 'lastName', 'firstName', 'extension', 'email',
       'officeCode', 'reportsTo', 'jobTitle'],
      dtype='object') EMPLOYEES


In [19]:
# Your code here
"""
products --> orderdetails ---> orders ---> customers ---> employees
"""

q = ("""
SELECT
    employees.employeeNumber AS "Employee Number",
    SUM(quantityOrdered) AS "Total Products Sold",
    employees.firstName || ' ' || employees.lastName AS "Employee Name"
FROM products
JOIN orderdetails
    ON products.productCode = orderdetails.productCode
JOIN orders
    ON orderdetails.orderNumber = orders.orderNumber
JOIN customers
    ON orders.customerNumber = customers.customerNumber
JOIN employees
    ON customers.salesRepEmployeeNumber = employees.employeeNumber
GROUP BY employees.employeeNumber
ORDER BY  employees.firstName || ' ' || employees.lastName
""")

pd.read_sql(q,conn)

,Employee Number,Total Products Sold,Employee Name
0,1611,6246,Andy Fixter
1,1504,7486,Barry Jones
2,1286,5016,Foon Yue Tseng
3,1323,7423,George Vanauf
4,1370,14231,Gerard Hernandez
5,1188,4227,Julie Firrelli
6,1501,8205,Larry Bott
7,1165,11854,Leslie Jennings
8,1166,4056,Leslie Thompson
9,1337,6186,Loui Bondur


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [29]:
# Your code here
"""
products --> orderdetails ---> orders ---> customers ---> employees
"""

q = ("""
SELECT
    COUNT(DISTINCT products.productCode) AS "Unique Products Sold",
    employees.employeeNumber AS "Employee Number",
    employees.firstName || ' ' || employees.lastName AS "Employee Name"
FROM products
JOIN orderdetails
    ON products.productCode = orderdetails.productCode
JOIN orders
    ON orderdetails.orderNumber = orders.orderNumber
JOIN customers
    ON orders.customerNumber = customers.customerNumber
JOIN employees
    ON customers.salesRepEmployeeNumber = employees.employeeNumber
GROUP BY employees.employeeNumber
HAVING COUNT(DISTINCT products.productCode) > 200
ORDER BY employees.firstName || ' ' || employees.lastName
""")


pd.read_sql(q,conn)

,Unique Products Sold,Employee Number,Employee Name


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!